In [1]:
import os
import pandas as pd
import PyPDF2
import re

# Paths
pdf_folder = r"D:\Swapnil's personal space\Research\System And Contextual information integration- Scii\Final Tender Code\Step Distribution\Step-3 Data_extraction\Step 3.1 Pdf_text_extraction\Technical specification extraction\pdf processed"
technical_doc_folder = r"D:\Swapnil's personal space\Research\System And Contextual information integration- Scii\Final Tender Code\Step Distribution\Step-3 Data_extraction\Step 3.1 Pdf_text_extraction\Technical specification extraction\technical doc"

# Function to extract PDF content
def extract_pdf_content(pdf_path):
    content = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text = page.extract_text()
            text = re.sub(r'[\u0900-\u097F]+', '', text)
                
            # Remove patterns like (cid:7414) along with the parentheses
            text = re.sub(r'\s*\(cid:\d+\)\s*', '', text)

            # Optionally, remove extra spaces left after cleaning
            text = re.sub(r'\s+', ' ', text).strip()

            content += text
            
    return content

# Get list of all PDF files and text files upfront
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
text_files = {f.replace('_tech_specs.txt', ''): f for f in os.listdir(technical_doc_folder)}

# Collect data
data = []
pdfs_with_matching_text_files = 0
missing_text_files = 0

for file_name in pdf_files:
    pdf_path = os.path.join(pdf_folder, file_name)
    base_name = file_name.replace('.pdf', '')
    
    # Check if corresponding text file exists in the preloaded dictionary
    if base_name in text_files:
        text_file_path = os.path.join(technical_doc_folder, text_files[base_name])
        pdfs_with_matching_text_files += 1

        # Read technical specification
        with open(text_file_path, 'r', encoding='utf-8') as text_file:
            technical_spec = text_file.read()

        # Extract PDF content
        pdf_content = extract_pdf_content(pdf_path)

        # Append to data
        data.append({
            'file_name': file_name,
            'content': pdf_content,
            'technical_specification': technical_spec
        })
    else:
        missing_text_files += 1

# Debug: Total counts
print(f"Total PDFs with matching text files: {pdfs_with_matching_text_files}")
print(f"Total PDFs without matching text files: {missing_text_files}")

# Create DataFrame
df = pd.DataFrame(data)

# Debug: Check DataFrame
print(df)


incorrect startxref pointer(1)


Total PDFs with matching text files: 760
Total PDFs without matching text files: 1
        file_name                                            content  \
0      file10.pdf  Tender For Fundus Fluorescein angiography with...   
1     file100.pdf  Tender For Auto Transfusion Machine (Cell Save...   
2    file1000.pdf  Tender No. AIIMS/R/CS/Neuro/1173/23/ GTE 1 vf[...   
3    file1001.pdf  GTE No.: AIIMS/ R/CS/ URO/22/17/GT Page 1 of 7...   
4    file1002.pdf  Tender No. AIIMS/R/CS/ ENT /1158 /22/ GTE /A 1...   
..            ...                                                ...   
755   file992.pdf  Tender No. AIIMS/R/CS/SG/Ech/1/8/2022/GTE 1 vf...   
756   file993.pdf  Tender No. AIIMS/R/CS/ OBGY /6795/2023/ GTE 1 ...   
757   file995.pdf  Tender No. AIIMS/R/CS/ Sur_Gastro/ Laproscopic...   
758   file997.pdf  Tender No. AIIMS/R/CS/ 4007/PHY PMR /23/ GTE /...   
759   file998.pdf  Tender No. AIIMS/R/CS/ Neuro /ICP/244/ GTE 1 v...   

                               technical_specificati

In [2]:
output_json_path = r"..\output_data.json"

# Export DataFrame to JSON file
df.to_json(output_json_path, orient='records', indent=4)